In [1]:
import sys
sys.path.append('../src/')
from vi_hrg_fixRTalpha import *
from utils import c2d, hyperdist, p_hd, polar2cart, warn_tensor
from torch import autograd
torch.manual_seed(4)

In [2]:
def noise_r(x, R, rel_var=0.1, epsilon=1e-4):
    rs = torch.distributions.normal.Normal(x, R*rel_var).sample() 
    return torch.clamp(rs, min=0+epsilon, max=R.item()-epsilon)

In [3]:
def noise_phi(x, rel_var=0.1):
    phis = torch.distributions.normal.Normal(x, 2*np.pi*rel_var).sample()
    return phis % (2*np.pi)

In [4]:
logit = lambda x: (x/(1-x)).log()

In [5]:
N = 75
R = torch.tensor([5.0]).double()
alpha = 1.1
T = 0.1

G = HRG(R=R, alpha=alpha, T=T)
r, theta, A = G.generate(N)
#G.show()
#G.plot()

In [6]:
no = noise_r(r, R, rel_var=0.2)/R
no.sum()

tensor(60.5496, dtype=torch.float64)

In [7]:
logit(no)

tensor([  2.6746,   2.9183,   1.3132,   0.6718,   1.2477,  -0.5310,   2.2771,
          1.7808,   1.5798,   0.8254,   2.1920,   0.5540,  10.8198,  10.8198,
         10.8198,  10.8198,   2.9095,  -0.1835,   0.7180,   0.8373,  10.8198,
          0.1590,   0.5464,   0.3894,   1.7198,  10.8198,  10.8198,  10.8198,
          5.2910,   0.6381,  10.8198,   2.4758,   0.2960,  10.8198,   0.9693,
          0.6524,   0.3752,  10.8198,  10.8198,   3.0857,  10.8198,   1.4461,
         10.8198,   0.8689,   1.2596,   5.1957,   2.3141,   2.1058,   1.2496,
          1.4135,   2.3126,   1.6948,  10.8198,  -0.0887, -10.8198,   2.5612,
          1.8634,  10.8198,  10.8198,   4.0200,   1.4320,  10.8198, -10.8198,
         10.8198,   2.3117,   1.1346,   0.7262,   0.8638,  10.8198,   1.4715,
         10.8198,   0.7079,   1.0100,   0.2759,  -0.1412], dtype=torch.float64)

In [8]:
r_loc_init = logit(noise_r(r, R, rel_var=0.2)/R)*R
r_scale_init = torch.ones([N]).double()
phi_loc_init = torch.cat((polar2cart(1, noise_phi(theta, 0.1)),  torch.zeros([N,1]).double()), dim=-1)
phi_loc_init.shape

torch.Size([75, 3])

In [9]:
r_loc_init

tensor([ 5.4099e+01,  3.5383e+00, -3.4698e+00,  3.8693e+00, -8.6157e+00,
         2.6317e+00,  5.4099e+01,  9.9548e+00, -4.6567e-01,  1.2732e+01,
         4.8921e+00, -1.6192e+00,  1.0127e+01,  6.5123e+00,  5.4099e+01,
         4.1010e+00,  5.4099e+01, -3.2952e+00,  1.0673e+01, -8.2810e-01,
         5.4099e+01,  5.5356e+00,  5.4099e+01,  1.1559e+00,  5.4099e+01,
         5.4099e+01,  1.0585e+01,  3.4178e-01,  1.1671e+01,  5.3209e+00,
         5.4099e+01,  1.0937e+01,  3.4221e+00,  1.3904e+01,  8.9842e+00,
        -5.3171e+00,  1.2897e+00,  5.4099e+01,  2.2897e+00,  1.7119e+01,
         1.1662e+00,  5.4099e+01,  1.4320e+01,  3.7737e+00,  4.4501e+00,
         7.7525e+00,  5.4099e+01,  2.7599e+01,  2.9212e+00,  1.0144e+01,
         1.2338e+00,  5.4099e+01,  3.8892e+00, -6.8353e+00, -1.6286e+00,
         5.4099e+01,  5.4099e+01,  5.4099e+01,  5.4099e+01,  1.0347e+01,
         1.1233e+00,  6.9845e+00, -5.2302e+00,  5.4099e+01,  6.0655e+00,
         1.1749e+01,  2.0045e+00, -2.2764e-02,  5.4

In [10]:
R_conc_init = torch.tensor(5.).log()
R_scale_init = torch.tensor(1.).log()
alpha_conc_init = torch.tensor(1.1).log()
alpha_scale_init = torch.tensor(.5).log()
T_init = torch.tensor([0.1,10.]).log()

In [11]:
dataloader = DataLoader(EdgesDataset(A), batch_size=128, shuffle=True, num_workers=0)
vi = VI_HRG(75,50, init_values={'rs_loc':r_loc_init,
                                'rs_scale':r_scale_init,
                              'phis_loc':phi_loc_init,
                              'phis_scale':None, 
                              'R_conc':R_conc_init, 
                              'R_scale':R_scale_init,
                              'alpha_conc':alpha_conc_init,
                              'alpha_scale':alpha_scale_init,
                              'T':T_init})
vi.dataloader = dataloader
vi.optimizer = torch.optim.RMSprop(vi.parameters())
#vi.optimizer.lr

In [14]:
total_loss = 0
with autograd.detect_anomaly():
    #print(torch.is_anomaly_enabled())
    for idx1, idx2, data in dataloader:
        loss = - vi.elbo(idx1, idx2, data, debug=True)
        vi.optimizer.zero_grad()
        loss.backward()
        vi.optimizer.step()
        print('>>>>', loss)
        total_loss += loss
total_loss        

Prob_edges  >> tensor(-265.7391, dtype=torch.float64, grad_fn=<SumBackward0>)
Alpha_R_ri  >> tensor(1.2856, dtype=torch.float64, grad_fn=<MulBackward0>)
Alpha       >> tensor(0.1627, dtype=torch.float64)
Alpha_R     >> tensor(0.0140, dtype=torch.float64)
P(q_ri)     >> tensor(175.0371, dtype=torch.float64, grad_fn=<MulBackward0>)
P(q_phii)   >> tensor(3.6811, dtype=torch.float64, grad_fn=<MulBackward0>)
>>>> tensor(88.6953, dtype=torch.float64, grad_fn=<NegBackward>)
Prob_edges  >> tensor(-309.5678, dtype=torch.float64, grad_fn=<SumBackward0>)
Alpha_R_ri  >> tensor(1.2475, dtype=torch.float64, grad_fn=<MulBackward0>)
Alpha       >> tensor(0.1627, dtype=torch.float64)
Alpha_R     >> tensor(0.0140, dtype=torch.float64)
P(q_ri)     >> tensor(214.5072, dtype=torch.float64, grad_fn=<MulBackward0>)
P(q_phii)   >> tensor(3.7603, dtype=torch.float64, grad_fn=<MulBackward0>)
>>>> tensor(93.0129, dtype=torch.float64, grad_fn=<NegBackward>)
Prob_edges  >> tensor(-220.7003, dtype=torch.float64, gr

Alpha_R_ri  >> tensor(1.0194, dtype=torch.float64, grad_fn=<MulBackward0>)
Alpha       >> tensor(0.1627, dtype=torch.float64)
Alpha_R     >> tensor(0.0140, dtype=torch.float64)
P(q_ri)     >> tensor(580.1964, dtype=torch.float64, grad_fn=<MulBackward0>)
P(q_phii)   >> tensor(3.7895, dtype=torch.float64, grad_fn=<MulBackward0>)
>>>> tensor(-327.3725, dtype=torch.float64, grad_fn=<NegBackward>)
Prob_edges  >> tensor(-229.3619, dtype=torch.float64, grad_fn=<SumBackward0>)
Alpha_R_ri  >> tensor(1.0306, dtype=torch.float64, grad_fn=<MulBackward0>)
Alpha       >> tensor(0.1627, dtype=torch.float64)
Alpha_R     >> tensor(0.0140, dtype=torch.float64)
P(q_ri)     >> tensor(978.3303, dtype=torch.float64, grad_fn=<MulBackward0>)
P(q_phii)   >> tensor(3.6414, dtype=torch.float64, grad_fn=<MulBackward0>)
>>>> tensor(-750.6804, dtype=torch.float64, grad_fn=<NegBackward>)
Prob_edges  >> tensor(-257.0310, dtype=torch.float64, grad_fn=<SumBackward0>)
Alpha_R_ri  >> tensor(1.0458, dtype=torch.float64, g

>>>> tensor(-1481.1945, dtype=torch.float64, grad_fn=<NegBackward>)
Prob_edges  >> tensor(-229.7128, dtype=torch.float64, grad_fn=<SumBackward0>)
Alpha_R_ri  >> tensor(1.0703, dtype=torch.float64, grad_fn=<MulBackward0>)
Alpha       >> tensor(0.1627, dtype=torch.float64)
Alpha_R     >> tensor(0.0140, dtype=torch.float64)
P(q_ri)     >> tensor(2108.1708, dtype=torch.float64, grad_fn=<MulBackward0>)
P(q_phii)   >> tensor(3.6577, dtype=torch.float64, grad_fn=<MulBackward0>)
>>>> tensor(-1880.2259, dtype=torch.float64, grad_fn=<NegBackward>)
Prob_edges  >> tensor(-278.0809, dtype=torch.float64, grad_fn=<SumBackward0>)
Alpha_R_ri  >> tensor(1.3374, dtype=torch.float64, grad_fn=<MulBackward0>)
Alpha       >> tensor(0.1627, dtype=torch.float64)
Alpha_R     >> tensor(0.0140, dtype=torch.float64)
P(q_ri)     >> tensor(1528.2272, dtype=torch.float64, grad_fn=<MulBackward0>)
P(q_phii)   >> tensor(3.7318, dtype=torch.float64, grad_fn=<MulBackward0>)
>>>> tensor(-1252.2554, dtype=torch.float64, gra

tensor(-29238.0845, dtype=torch.float64, grad_fn=<AddBackward0>)

In [13]:
total_loss

tensor(10253.7785, dtype=torch.float64, grad_fn=<AddBackward0>)

In [12]:
with autograd.detect_anomaly():
    vi.train(dataloader, lrs=0.01)

>>>>>>>>>>>> Start training...


../src/utils.py:126: UserWarning: cosh_dist has 1 in it!
  warnings.warn(str('%s has 1 in it!' % variable))


Epoch 1 | LR: 0.01 | Total loss: 10253.78 | Epoch time 4.79
Epoch 2 | LR: 0.01 | Total loss: -29238.08 | Epoch time 4.68
Epoch 3 | LR: 0.01 | Total loss: -477261.05 | Epoch time 4.82
Epoch 4 | LR: 0.01 | Total loss: -5646630.46 | Epoch time 4.96


KeyboardInterrupt: 

In [ ]:
r_loc_init

In [ ]:
vi.alpha_conc.exp(), vi.alpha_scale.exp()

In [15]:
vi.rs_loc, vi.rs_scale.exp()

(Parameter containing:
 tensor([56.9091,  3.9635, -3.9122,  3.8555, -8.3715,  1.5815, 56.8032,  9.0537,
          0.8439, 11.6298,  5.0097, -1.3852,  9.2850,  6.9896, 56.5487,  4.5372,
         56.6321, -3.3019, 10.2364, -1.4149, 56.7479,  5.9424, 56.7463,  2.0818,
         56.7598, 56.4430,  9.3889,  1.5549, 10.5041,  5.6808, 56.5357,  9.9711,
          3.8809, 12.5731,  8.3478, -5.2546,  1.8356, 56.4930,  2.4963, 15.9257,
          2.3456, 56.7916, 13.1642,  3.2455,  4.0737,  7.5471, 56.7527, 26.3773,
          3.0373,  9.1823,  1.4093, 56.4141,  4.2777, -6.5819, -2.8102, 56.7080,
         56.9363, 56.8791, 56.7144,  9.2963,  0.5525,  7.1836, -5.6121, 56.8700,
          6.3156, 10.7857,  2.5535,  1.0002, 56.8336, 17.0014, 56.6810, -2.4243,
         -5.2065,  2.9432,  4.8724], dtype=torch.float64, requires_grad=True),
 tensor([0.1596, 1.5768, 1.6087, 2.2316, 2.1865, 6.5008, 0.1776, 2.4938, 3.5170,
         2.9182, 1.9397, 3.1396, 2.3758, 1.5585, 0.2289, 1.6529, 0.2104, 2.4660,
       

In [16]:
Radius(vi.rs_loc, vi.rs_scale.exp(), torch.ones([N])*R).mean

tensor([5.0000e+00, 4.9068e+00, 9.8019e-02, 4.8964e+00, 1.1565e-03, 4.1471e+00,
        5.0000e+00, 4.9994e+00, 3.4965e+00, 5.0000e+00, 4.9669e+00, 1.0009e+00,
        4.9995e+00, 4.9954e+00, 5.0000e+00, 4.9471e+00, 5.0000e+00, 1.7753e-01,
        4.9998e+00, 9.7734e-01, 5.0000e+00, 4.9869e+00, 5.0000e+00, 4.4456e+00,
        5.0000e+00, 5.0000e+00, 4.9996e+00, 4.1281e+00, 4.9999e+00, 4.9830e+00,
        5.0000e+00, 4.9998e+00, 4.8989e+00, 5.0000e+00, 4.9988e+00, 2.5983e-02,
        4.3121e+00, 5.0000e+00, 4.6194e+00, 5.0000e+00, 4.5629e+00, 5.0000e+00,
        5.0000e+00, 4.8126e+00, 4.9163e+00, 4.9974e+00, 5.0000e+00, 5.0000e+00,
        4.7712e+00, 4.9995e+00, 4.0183e+00, 5.0000e+00, 4.9316e+00, 6.9168e-03,
        2.8387e-01, 5.0000e+00, 5.0000e+00, 5.0000e+00, 5.0000e+00, 4.9995e+00,
        3.1736e+00, 4.9962e+00, 1.8201e-02, 5.0000e+00, 4.9910e+00, 4.9999e+00,
        4.6390e+00, 3.6555e+00, 5.0000e+00, 5.0000e+00, 5.0000e+00, 4.0668e-01,
        2.7256e-02, 4.7497e+00, 4.9620e+

In [ ]:
vi.r_i_samples

In [ ]:
vi.R_conc.exp(), vi.R_scale.exp()

In [ ]:
p_hd_ = lambda d,R,T: (1.+((d-R)/(2.*T)).exp()).reciprocal()
phd = lambda d,R,T: 0.5 + 0.5*(-(d-R)/(4*T)).tanh()

In [ ]:
p_hd_(torch.tensor(-np.inf),R,T)

In [ ]:
phd(torch.tensor(5.1),R,T)

In [ ]:
torch.tensor([0.1,10.0]).log()

In [ ]:
x = torch.arange(0.,1.,.01)
plt.plot(x, torch.sigmoid(logit(x)))
plt.show()

In [ ]:
logit(torch.tensor(0.))

In [ ]:
l, n, m = 3, 4, 5
broadcast = torch.ones([l,n])

In [ ]:
torch.rand(m).expand(n,m).t()

In [ ]:
torch.arange(n).expand(m,n)

In [ ]:
torch.tensor(0.).mean()

In [ ]:
arcosh = lambda x: (torch.clamp(x, min=1.) + (torch.clamp(x, min=1.)**2 - 1).sqrt())
arcosh(torch.arange(0.,2., .1))

In [ ]:
torch.clamp(x, min=1.)